In [ ]:
import math
import numpy as np
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import scipy
from scipy import stats
from scipy.stats import t
from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
import matplotlib.pyplot as plt

Code used for breakpoint analysis.

Google Drive Folder with data: https://drive.google.com/drive/folders/13KgY7vxar1qEAdFZqI91m9O2sgiC0o7_?usp=share_link

Note: Some codes are file format sensitive (csv/xlsx)

In [ ]:
path = "/content/drive/MyDrive/multi multi easy/Bacteria.csv"
df = pd.read_csv(path)
df = df.fillna(0)
df

In [ ]:
!pip install piecewise-regression

In [ ]:
x = df['Total']
y = df['C']

y = y[y != 0] # Remove zero values
x = x[y.index] # Select corresponding x values
logx = np.log(x)
logy = np.log(y)

In [ ]:
import piecewise_regression
pw_fit = piecewise_regression.Fit(np.array(logx), np.array(logy), n_breakpoints=1)
pw_fit.summary()

In [ ]:
bp = pw_fit.get_results()['estimates']['breakpoint1']['estimate']

x = np.array(x)
y = np.array(y)

x1 = x[x < np.exp(bp)]
y1 = y[np.argwhere(x < np.exp(bp)).squeeze()]
x2 = x[(x > np.exp(bp))]
y2 = y[np.argwhere((x > np.exp(bp))).squeeze()]

In [ ]:
np.exp(bp)

In [ ]:
def power_law(x, a, b):
  """Probability Density Function (PDF) for Powerlaw."""
  return a * np.power(x, b)

def parameters(x, y, pars, cov):
  # Pars
  logx = np.log(x)
  logy = np.log(y)

  stdevs = np.sqrt(np.diag(cov))
  res_plaw = y - power_law(x, *pars) # Residual for power law fit
  res = stats.linregress(logx, logy)
  tinv = lambda p, df: abs(t.ppf(p/2, df))
  ts = tinv(0.05, len(logx) - 2)
  std_err = res.stderr
  b, loga = np.polyfit(logx, logy, 1)
  a = np.exp(loga)
  return b, ts * std_err, a

#fig, ax = plt.subplots(1, 1)
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
pars1, cov1 = curve_fit(f=power_law, xdata=x1, ydata=y1, p0=[0, 0], bounds=(-np.inf, np.inf), maxfev = 2000)
slope1, slope_err1, a1 = parameters(x1, y1, pars1, cov1)
ax.plot(x1, power_law(x1, *pars1), linestyle='-', linewidth=1, color='#fcbba1');

pars2, cov2 = curve_fit(f=power_law, xdata=x2, ydata=y2, p0=[0, 0], bounds=(-np.inf, np.inf), maxfev = 4000)
slope2, slope_err2, a2 = parameters(x2, y2, pars2, cov2)
ax.plot(x2, power_law(x2, *pars2), linestyle='-', linewidth=1, color='#969696');

sns.scatterplot(x=x1, y=y1, s=10, marker="o", color='#fcbba1')
sns.scatterplot(x=x2, y=y2, s=10, marker="o", color='#969696')

#FC9272
#009E73
plt.style.use('seaborn')
plt.style.use('tableau-colorblind10')
sns.set_style('whitegrid')
sns.set_context("paper")
ax.legend(loc=0, fontsize=14);
#ax.set_xlabel("Total protein annotations", fontsize=14);
#ax.set_ylabel("Category annotations", fontsize=14);
ax.set_title("[E] Amino acid transport and \n metabolism", fontsize=12);
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim([1e2, 1e4])
plt.savefig('14 ar E', bbox_inches='tight', dpi=500)

print(f"Slope 1: {slope1:.2f} +/- {slope_err1:.2f}")
print(f"Slope 2: {slope2:.2f} +/- {slope_err2:.2f}")
#; Normalization constant:{a1}"
#; Normalization constant:{a2}